<a href="https://colab.research.google.com/github/Ry02024/SIGNATE/blob/main/%E3%82%A2%E3%83%A4%E3%83%A1%E3%81%AE%E5%88%86%E9%A1%9E/%E3%83%87%E3%83%BC%E3%82%BF%E3%82%B5%E3%82%A4%E3%82%A8%E3%83%B3%E3%82%B9%E3%82%A2%E3%83%97%E3%83%AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#streamlit：簡単な分析と機械学習

##環境構築

In [ ]:
!pip install streamlit -q
!pip install pyngrok -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.9 MB/s eta 0:00:00


In [ ]:
import streamlit as st
from pyngrok import ngrok

In [ ]:
from google.colab import userdata
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')

In [ ]:
from pyngrok import ngrok
import os

# ngrokトークンを設定
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

##実行ファイル

###version1

In [ ]:
!pip install japanize-matplotlib -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


###util

In [75]:
%%writefile util.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
import google.generativeai as genai

def load_data(file, drop_id=True):
    """
    ファイルをアップロードし、データフレームを読み込む。
    """
    if file is not None:
        data = pd.read_csv(file, delimiter='\t')
        if drop_id and 'id' in data.columns:
            data.drop(columns='id', inplace=True)
        return data
    return None

def visualize_data_analysis(data):
    """
    データ分析の可視化を実行する。
    """
    if data is not None:
        st.write('基本統計量:')
        st.write(data.describe())

        st.write('特徴量のヒストグラム:')
        fig, ax = plt.subplots()
        data.hist(bins=15, figsize=(15, 10), ax=ax)
        st.pyplot(fig)

        st.write('散布図とカーネル密度推定:')
        pair_plot_fig = sns.pairplot(data, hue='class', markers=["o", "s", "D"], palette="bright")
        st.pyplot(pair_plot_fig)

        st.write('相関行列:')
        numeric_data = data.select_dtypes(include=['number'])
        fig, ax = plt.subplots()
        sns.heatmap(numeric_data.corr(), annot=True, cmap='coolwarm', ax=ax)
        st.pyplot(fig)

def train_and_evaluate_model(train_data, test_data):
    """
    モデルを訓練し、評価指標を表示する。
    """
    if train_data is not None and test_data is not None:
        # 特徴量とターゲットに分割
        X_train = train_data.drop('class', axis=1)
        y_train = train_data['class']
        X_test = test_data

        # 特徴量のスケーリング
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # モデル構築とハイパーパラメータのチューニング
        model = RandomForestClassifier(random_state=42)
        param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20, 30]}
        grid_search = GridSearchCV(model, param_grid, cv=5)
        grid_search.fit(X_train_scaled, y_train)

        # 最適なモデルを取得
        best_model = grid_search.best_estimator_

        # 訓練データに対する予測
        y_train_pred = best_model.predict(X_train_scaled)
        train_accuracy = accuracy_score(y_train, y_train_pred)
        train_confusion = confusion_matrix(y_train, y_train_pred)

        # 適合率、再現率、F1スコアを計算
        precision = precision_score(y_train, y_train_pred, average='macro')
        recall = recall_score(y_train, y_train_pred, average='macro')
        f1 = f1_score(y_train, y_train_pred, average='macro')

        st.write("訓練データの精度: ", train_accuracy)
        st.write("適合率:", precision)
        st.write("再現率:", recall)
        st.write("F1スコア:", f1)
        fig, ax = plt.subplots()
        sns.heatmap(train_confusion, annot=True, fmt='d', cmap='Blues', ax=ax)
        plt.title("訓練データの混同行列")
        st.pyplot(fig)

        # テストデータに対する予測
        y_test_pred = best_model.predict(X_test_scaled)
        st.write("テストデータの予測結果:")
        st.write(pd.DataFrame({'Prediction': y_test_pred}))

        # 予測結果の要約を生成
        prediction_summary = f"""
        訓練データの精度: {train_accuracy}
        適合率: {precision}
        再現率: {recall}
        F1スコア: {f1}
        """
        return prediction_summary, y_test_pred
    return None, None

def get_insights_from_gemini(summary, y_test_pred):
    """
    Gemini APIを使用して、予測結果に基づいた知見を取得する。
    """
    model = genai.GenerativeModel('gemini-pro')
    combined_text = f"""
    以下の予測結果に基づいて、モデルのパフォーマンスと結果についての詳細な分析を提供してください。

    **予測結果の要約:**
    {summary}

    **テストデータの予測結果:**
    {y_test_pred}
    """
    response = model.generate_content(combined_text)
    return response.text


Overwriting util.py


In [76]:
%%writefile streamlit_app.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
from util import load_data, visualize_data_analysis, train_and_evaluate_model, get_insights_from_gemini

st.title('アヤメの品種分類アプリ')
st.write('このアプリはアヤメのデータセットを分析し、品種を予測します。')

# APIキー入力部分
api_key = st.text_input("APIキーを入力してください:", value="", type="password")

if api_key:
    import google.generativeai as genai
    genai.configure(api_key=api_key)

# ファイルアップローダー
train_file = st.file_uploader("訓練データをアップロードしてください（train.tsv）", type=['tsv'])
test_file = st.file_uploader("テストデータをアップロードしてください（test.tsv）", type=['tsv'])

if train_file is not None and test_file is not None:
    train_data = load_data(train_file)
    test_data = load_data(test_file, drop_id=True)

    # データの表示
    st.write("訓練データの最初の5行:")
    st.write(train_data.head())
    st.write("テストデータの最初の5行:")
    st.write(test_data.head())

    # 基本統計と分布の表示
    if st.checkbox('基本統計と分布を表示'):
        visualize_data_analysis(train_data)

    # 散布図とカーネル密度推定
    if st.checkbox('散布図とカーネル密度推定を表示'):
        pair_plot_fig = sns.pairplot(train_data, hue='class', markers=["o", "s", "D"], palette="bright")
        st.pyplot(pair_plot_fig)

    # 相関行列
    if st.checkbox('相関行列を表示'):
        numeric_data = train_data.select_dtypes(include=['number'])
        fig, ax = plt.subplots()
        sns.heatmap(numeric_data.corr(), annot=True, cmap='coolwarm', ax=ax)
        st.pyplot(fig)

    # モデルの訓練と評価
    if st.button('モデルを訓練して評価'):
        summary, y_test_pred = train_and_evaluate_model(train_data, test_data)
        if summary:
            st.write("モデルのパフォーマンス要約:")
            st.write(summary)

            # Geminiを使って知見を取得
            insights = get_insights_from_gemini(summary, y_test_pred)
            st.write("Geminiからの知見:")
            st.write(insights)


Overwriting streamlit_app.py


##アプリの起動

In [77]:
# ngrokを介してStreamlitを公開
public_url = ngrok.connect(addr='8501')
print('Public URL:', public_url)

Public URL: NgrokTunnel: "https://a4d5-34-136-117-121.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
# Streamlitアプリケーションの起動
!streamlit run streamlit_app.py >/dev/null

/content/util.py:33: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared.
  data.hist(bins=15, figsize=(15, 10), ax=ax)
/content/util.py:33: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared.
  data.hist(bins=15, figsize=(15, 10), ax=ax)
/content/util.py:33: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared.
  data.hist(bins=15, figsize=(15, 10), ax=ax)
